In [ ]:
import sys

sys.path.append('..//scripts//')

In [ ]:
%matplotlib notebook

# allows for automatic reloading of imports and makes it unncessecary to restart the kernel
# whenever a function is changed
%load_ext autoreload
%autoreload 2

import os
import time
import numpy as np
import csv
import pandas as pd

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from constants.paths import CENTRAL_GROUPS_CSV, RADII_CSV, METHYL_CSV

from classes.Settings import AlignmentSettings, Radii

from helpers.alignment_helpers import (calc_rmse, kabsch_align, perform_rotations,
                                       perform_translation, read_raw_data)
from align_kabsch import align_all_fragments

from helpers.geometry_helpers import make_coordinate_df, average_fragment, add_model_methyl
from helpers.density_helpers import prepare_df

central_groups = ["REt"]
contact_groups = ["CF", "R2CO"]
to_count =       ["F", "O"]

In [ ]:
def rmse_test(avg_fragment, settings):          
    rmse_list = []
       
    frags_file, structures_file = settings.get_aligned_csv_filenames()
    
    avg_rmse = pd.read_csv(structures_file).rmse.mean()
    print(f"Average RMSE kabsch alignment: {avg_rmse}")
    
    # this is the old number of atoms
    rows = 1000 *  settings.no_atoms

    df = pd.read_csv(frags_file, nrows=rows)
    
    # correct after reading
    settings.no_atoms = len(df[df.fragment_id == 0])
    settings.no_atoms_central = len(df[(df.fragment_id == 0) & (df.label != "-")])
    print(settings.no_atoms, settings.no_atoms_central)
        
    rows = 2 *  settings.no_atoms
    df = pd.read_csv(frags_file, nrows=rows)
    
    unique_frags = df.fragment_id.unique()
    print(len(unique_frags))
    
    # matrix A is avg fragment
    avg_fragment = avg_fragment[~(avg_fragment.label.str.contains('aH')) & (avg_fragment.label != "-")]
    print(avg_fragment)
    A = np.array([np.array(avg_fragment.x), np.array(avg_fragment.y), np.array(avg_fragment.z)]).T
    n = A.shape[0]
        
    # matrix B is first 100 fragments
    for unique_frag in unique_frags:
        frag = df[df.fragment_id == unique_frag][:settings.no_atoms_central]
        B = np.array([np.array(frag.x), np.array(frag.y), np.array(frag.z)]).T
        
        rmse_list.append(calc_rmse(A, B, n))
    
    print(A, B)
        
    print(len(rmse_list))
    print(rmse_list[:10])
        
    return rmse_list

In [ ]:
for central_group in central_groups:
    for to_count_contact, contact_group in zip(to_count, contact_groups):

        print('\n', central_group, contact_group, to_count_contact)

        datafile = "..\\data\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5.cor"
        result1 = "..\\..\\results\\pairs\\" + central_group + "\\" + central_group + "_" + contact_group + "_vdw.5\\"\
                  + central_group + "_" + contact_group + "_aligned.csv" 

        if not os.path.exists(f"..\\..\\results\\pairs\\{central_group}"):
            os.mkdir(f"..\\..\\results\\pairs\\{central_group}\\")

        t0_alignment = time.time()

        labelfile = datafile.rsplit('.', 1)[0] + '.csv'

        settings = AlignmentSettings("..\\..", datafile, labelfile)
        settings.set_atom_to_count(to_count_contact)
        settings.set_central_group_csv(CENTRAL_GROUPS_CSV)
        settings.prepare_alignment()

        align_all_fragments(settings)

        aligned_fragments_df = pd.read_csv(settings.get_aligned_csv_filename())

#         radii = Radii(RADII_CSV)
#         avg_frag = average_fragment(aligned_fragments_df, settings, radii)
        avg_frag = pd.read_csv(settings.get_avg_frag_filename())
    
#         # DO RMSE TEST
        rmses = rmse_test(avg_frag, settings)
#         print(rmses)
        print(f"Average RMSE average fragment: {np.mean(rmses)}")
        
        
#         # IF FAILS